In [1]:
from selenium import webdriver
import datetime
import urllib
import requests
import re
import time

In [30]:
class News_parse:
    
    
        
    
    def str_date_to_int(date_str):
        y = int(date_str[:4])
        m = int(date_str[5:7])
        d = int(date_str[8:10])
        h = int(date_str[11:13])
        mi = int(date_str[14:16])

#         print("date slicing...")

        return y,m,d,h,mi


    
    def bubbleSort(alist):
        for passnum in range(len(alist)-1,0,-1):
            for i in range(passnum):
                if alist[i]<alist[i+1]:
                    temp = alist[i]
                    alist[i] = alist[i+1]
                    alist[i+1] = temp

    def content_to_str(body_content):
        body_str = str(body_content)

        sign_counter = 0
        start_sign_location = 0
        end_sign_location = 0
        para_counter =0
        for x in range(len(body_str)):
            if body_str[x] == "=":
                sign_counter += 1
            if sign_counter == 2 :
                sign_counter = x
                start_sign_location = sign_counter
                for w in range(x,len(body_str)):
                    if body_str[w] == "\n":
                        para_counter += 1
                        if para_counter == 4 :
                            end_sign_location = w
                            break
                break
#         print(start_sign_location)        
#         print(end_sign_location)
        return body_str[start_sign_location+1:end_sign_location+1]
    
    def line_ele(strings):
    
        return strings.replace("\n"," ")
    
        

    def Parse(self):
        
        dt=datetime.datetime.now()
        target_date = datetime.datetime(dt.year,dt.month,dt.day-1,20,0)
        
        driver = webdriver.Chrome("C:\\Users\\infomax\\chromedriver_win32\\chromedriver")
        driver.implicitly_wait(2)
        driver.get("http://news.einfomax.co.kr/news/searchForm.html")

        article_list = {}

        name_list = ["오진우","곽세연","우성문"]
        target_date = target_date

        art_counts_1 =0

        driver.find_element_by_xpath("""//*[@id="search-tabs1"]/form/fieldset[2]/div[2]/div/label[4]""").click()

        for name in name_list:
            #이름입력
            driver.find_element_by_xpath("""//*[@id="sc_word"]""").send_keys(name)


            #검색 클릭
            driver.find_element_by_xpath("""//*[@id="search-tabs1"]/form/footer/div/button""").click()
             # 리스트형 클릭
            driver.find_element_by_xpath("""//*[@id="user-container"]/div[3]/div[2]/section/article/div[2]/header/div[2]/a[1]""").click()


            # 탑 기사 부터 처리 
            try:
                top_date = driver.find_element_by_xpath("""//*[@id="user-container"]/div[3]/div[2]/section/article/div[2]/section/div[1]/div/div[2]""")
                top_date.get_attribute('innerHTML')
                top_date_str = top_date.get_attribute('innerHTML')
                print(top_date_str)
                top_date_str2 =  top_date_str[-16:].strip()

                y,m,d,h,mi = News_parse.str_date_to_int(top_date_str2)

                top_date_value = datetime.datetime(y,m,d,h,mi)

                if top_date_value > target_date:
                    #탑에 걸려있는 기사에 접근
                    driver.find_element_by_xpath("""//*[@id="user-container"]/div[3]/div[2]/section/article/div[2]/section/div[1]/div/div[1]/a/strong""").click()
                    targetURL = driver.current_url
                    print(targetURL)

                    article_head = driver.find_element_by_xpath("""//*[@id="user-container"]/div[3]/header/div/div""").text
                    article_body_before =driver.find_element_by_xpath("""//*[@id="article-view-content-div"]""").text
                    article_body_after = News_parse.content_to_str(article_body_before)

                    art_counts_1 += 1
#                     print("기사개수: ",art_counts_1)

                    print(article_head)
#                     print(article_body_after)
                    article_list[top_date_value] = [article_head,article_body_after,targetURL]
                    driver.back()

            except ValueError: 
                print(name,"탑기사 오류 확인할것")



            for art_num in range(1, 15):
                #날짜 파싱하기 - 바디기사 날짜 파싱 
                time.sleep(2)
                body_date = driver.find_element_by_xpath("""//*[@id="user-container"]/div[3]/div[2]/section/article/div[2]/section/div[2]/div["""+str(art_num)+"""]/div[2]""")
                body_date.get_attribute('innerHTML')
                body_date_str = body_date.get_attribute('innerHTML')
#                 print(body_date_str)
                body_date_str2 = body_date_str[-16:].strip()

                y,m,d,h,mi = News_parse.str_date_to_int(body_date_str2)
                body_date_value = datetime.datetime(y,m,d,h,mi)
                body_date_value


                if body_date_value > target_date :
                #바디에 있는 기사에 접근
                    driver.find_element_by_xpath("""//*[@id="user-container"]/div[3]/div[2]/section/article/div[2]/section/div[2]/div["""+str(art_num)+"""]/div[1]/a/strong""").click()

                    targetURL = driver.current_url

                    art_counts_1 += 1
                    print("기사개수: ",art_counts_1)

                    print(targetURL)

                    article_head = driver.find_element_by_xpath("""//*[@id="user-container"]/div[3]/header/div/div""").text
                    article_body_before =driver.find_element_by_xpath("""//*[@id="article-view-content-div"]""").text
                    article_body_after = News_parse.content_to_str(article_body_before)
                    print(article_head)
#                     print(article_body_after)
                    article_list[body_date_value] = [article_head,article_body_after,targetURL]


                    driver.back()
            driver.back()
            driver.back()
            driver.find_element_by_xpath("""//*[@id="sc_word"]""").clear()
            
            article_to_list=list(article_list.keys())
            
        print("\n\n\n담긴 기사개수 : ",len(article_to_list))
        print("원래 기사개수 : ", art_counts_1)

        News_parse.bubbleSort(article_to_list)
        temp_art = []
        full_article = ""

        for y in article_to_list:
            temp_art = article_list[y]
            full_article = full_article + "    ▲" +temp_art[0]+"\n"+ "    -" + News_parse.line_ele(temp_art[1]) + "\n    " + temp_art[2] +"\n\n" 
        print("\n\n --------------------이하 복사--------------------","\n\n\n")
        print(full_article)

            
    

In [31]:
News= News_parse()

In [32]:
News.Parse()

국제경제 | 오진우 기자 | 2019-01-20 07:00
국제경제 | 곽세연 기자 | 2019-01-23 07:21
http://news.einfomax.co.kr/news/articleView.html?idxno=4013091
<뉴욕전문가 시각> 다시 돌아볼 때…무역·성장 우려 여전
기사개수:  2
http://news.einfomax.co.kr/news/articleView.html?idxno=4013084
뉴욕증시, 글로벌 성장 둔화 우려…다우 1.22% 하락 마감
국제경제 | 우성문 기자 | 2019-01-23 06:42
http://news.einfomax.co.kr/news/articleView.html?idxno=4013085
"지정학적 문제, 경제에 주요 위험"…다보스포럼 한목소리
기사개수:  4
http://news.einfomax.co.kr/news/articleView.html?idxno=4013078
"미, 중국과 무역협상 회의 취소"(상보)
담긴 기사개수 :  4
원래 기사개수 :  4
---------이하 복사-------------------- 



    ▲<뉴욕전문가 시각> 다시 돌아볼 때…무역·성장 우려 여전
    - 월가 전문가들은 22일 무역 긴장과 경제 성장 둔화는 여전하다며 반등했던 지수가 숨고르기를 할 것이라고 예상했다.  존스트레이딩의 마이클 오로크 수석 시장 전략가는 "많은 부정적인 소식을 소화하고 있다"며 "미국과 중국은 지적재산권에서 진전을 이루지 못했고 중국은 2025 전략을 강행하고 있다는 점에 주목했다"고 말했다.  
    http://news.einfomax.co.kr/news/articleView.html?idxno=4013091

    ▲"지정학적 문제, 경제에 주요 위험"…다보스포럼 한목소리
    - 스위스에서 열린 세계경제포럼, 다보스 포럼에서 참가자들이 조심스러운 경제 전망을 내놨다.  다보스 포럼에 참석한 각국 지도자들과 최고경영자(CEO), 은행관계자들은 정치적 여러 이슈 등이 투자

In [14]:
driver.close()

TypeError: str_date_to_int() missing 1 required positional argument: 'date_str'

In [18]:
class test:
    def a(x):
        print(x)
    def b(self):
        test.a("xxaa")

In [19]:
test1 = test()

In [20]:
test1.b()

xxaa
